In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import random
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import requests
import os
import time
import random
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
from threading import Thread, Lock
import threading
import pickle
import datetime

import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\src")

### РИА новости

In [20]:
# Запускаем функцию для сбора ссылок, останавливаем вручную, потому что нельзя листать ее до бесконечности.
from parsing import get_source_html_ria

driver = webdriver.Chrome(ChromeDriverManager().install())

get_source_html_ria(url='https://ria.ru/politics/', driver=driver)

/var/folders/m6/2n44mlkj7nbdwb651dd78pwc0000gn/T/ipykernel_810/1943467916.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


KeyboardInterrupt: 

In [37]:
# Вытаскиваем ссылки
from parsing import get_file_urls_ria
file_path = '/Users/eugenborisenko/Desktop/Универ мага/Диссертация/Парсеры сайтов/ria_politics/raw_preview'
get_file_urls_ria(file_path)

Ссылки из файлов 1 из 23 получены
Ссылки из файлов 2 из 23 получены
Ссылки из файлов 3 из 23 получены
Ссылки из файлов 4 из 23 получены
Ссылки из файлов 5 из 23 получены
Ссылки из файлов 6 из 23 получены
Ссылки из файлов 7 из 23 получены
Ссылки из файлов 8 из 23 получены
Ссылки из файлов 9 из 23 получены
Ссылки из файлов 10 из 23 получены
Ссылки из файлов 11 из 23 получены
Ссылки из файлов 12 из 23 получены
Ссылки из файлов 13 из 23 получены
Ссылки из файлов 14 из 23 получены
Ссылки из файлов 15 из 23 получены
Ссылки из файлов 16 из 23 получены
Ссылки из файлов 17 из 23 получены
Ссылки из файлов 18 из 23 получены
Ссылки из файлов 19 из 23 получены
Ссылки из файлов 20 из 23 получены
Ссылки из файлов 21 из 23 получены
Ссылки из файлов 22 из 23 получены
Ссылки из файлов 23 из 23 получены
Ссылки сохранены


In [2]:
# Загружаем сохраненные ссылки
with open('ria_politics/urls.txt') as file:
    urls = file.read()
urls = urls.split('\n')

In [2]:
# Создаем набор user agent, чтобы сайт не так быстро блочил
software_names = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   

user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)

In [3]:
# Создаем массив данных, куда будут все склвдывать потоки
data = {'link': [], 'data_or_ex': []}
total_news = len(urls)
current_value = 0

NameError: name 'urls' is not defined

In [ ]:
# Запускаем функцию парсинга
from parsing import start_parsing, parse_from_link_ria
start_parsing('ria_economy/raw_cites', parse, parse_from_link_ria, urls, n_threads=2, save_evry=200)

### Лента

In [2]:
software_names = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   

user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)

In [ ]:
from parsing import get_lenta_urls
get_lenta_urls(root='lenta_economy')

2018-01 готов
2018-02 готов
2018-03 готов
2018-04 готов
2018-05 готов
2018-06 готов


In [ ]:
with open('lenta_economy/urls.txt') as file:
    urls = file.read()
urls = urls.split('\n')

In [ ]:
data = {'link': [], 'data_or_ex': []}
total_news = len(urls)
current_value = 0

In [ ]:
from parsing import parse_from_link_lenta
start_parsing('lenta_economy/raw_cites', parse, parse_from_link_lenta, urls, n_threads=2, save_evry=200)

### Ведомости

In [4]:
from parsing import get_file_urls
get_file_urls('vedomosti_business')

Ссылки из файлов 1 из 1 получены
Ссылки сохранены


In [4]:
software_names = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   

user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)

In [12]:
with open('vedomosti_invest/urls_invest.txt') as file:
    urls = file.read()
urls = urls.split('\n')

In [13]:
data = {'link': [], 'data_or_ex': []}
total_news = len(urls)
current_value = 0

In [15]:
from parsing import parse_from_link_vedomosti
start_parsing('vedomosti_invest/raw_cites', parse, parse_from_link_vedomosti, urls, n_threads=2, save_evry=200)

0 saved

Invalid URL '': No scheme supplied. Perhaps you meant http://?


### Парсинг цен акций с Финам

In [61]:
companies = {'ALRS': 'alrosa-ao', 'AFLT': 'aeroflot', 'VTBR': 'vtb', 'GAZP' : 'gazprom', 'GMKN': 'nornickel-gmk', 'LSRG': 'lsr',
             'DSKY': 'detsky-mir', 'IRAO': 'inter-rao-ao', 'LKOH': 'lukoil', 'MAGN': 'mmk', 'MOEX': 'moscowexchange', 'CBOM': 'oao-mkb-ao', 
             'MTSS': 'mts', 'VKCO': 'gdr-vk-company-limited-ord-shs_vkco-tqbr', 'ROSN': 'rosneft', 'NLMK': 'nlmk-ao', 'NVTK': 'novatek',
             'PIKK': 'pik', 'PLZL': 'polus-zoloto', 'RTKM': 'rostelecom', 'RUAL': 'united-company-rusal-plc', 'HYDR': 'rusgidro',
             'SBER': 'sberbank', 'CHMF': 'severstal-ao', 'AFKS': 'afk-sistema', 'SNGS': 'surgut', 'TATN': 'tatneft-3', 'TCSG': 'gdr-tcs-group-holding-ord-shs_tcsg',
             'TRNFP': 'transneft-pref', 'POGR': 'petropavlovsk-plc_pogr', 'PHOR': 'phosagro-ao', 'FEES': 'fsk-ees', 'HHRU': 'adr-headhunter-group-ord-shs_hhru', 
             'YNDX': 'pllc-yandex-n-v'}

In [116]:
# link = 'https://www.finam.ru/profile/moex-akcii/pllc-yandex-n-v/export/'
from parsing import get_all_quotes
get_all_quotes(companies, year_start=2010)

/var/folders/m6/2n44mlkj7nbdwb651dd78pwc0000gn/T/ipykernel_1959/966038876.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(),


ALRS_100101_110101.csv
Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=109.0.5414.87)
Stacktrace:
0   chromedriver                        0x000000010462dfa8 chromedriver + 4886440
1   chromedriver                        0x00000001045ab643 chromedriver + 4351555
2   chromedriver                        0x00000001041f9b27 chromedriver + 477991
3   chromedriver                        0x00000001041cf0a5 chromedriver + 303269
4   chromedriver                        0x00000001042698ef chromedriver + 936175
5   chromedriver                        0x000000010427ead3 chromedriver + 1022675
6   chromedriver                        0x00000001042643a3 chromedriver + 914339
7   chromedriver                        0x000000010422e57f chromedriver + 693631
8   chromedriver                        0x000000010422fb1e chromedriver + 699166
9   chromedriver                        0x00000001045fab9e chromedriver + 4676510
10  chromedriver   

/var/folders/m6/2n44mlkj7nbdwb651dd78pwc0000gn/T/ipykernel_1959/966038876.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(),


KeyboardInterrupt: 

### Комерсант

In [2]:
software_names = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   

user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=1000)

In [14]:
# rubrics = {
#     #'2': 'politics', '3': 'economics', 
#     '4': 'business', '40':'finance', '41': 'consumer_market'}
# for rub in rubrics.values():
#     os.mkdir(f'komersant_{rub}')

In [8]:
#По номерам рубрик скачиваем ссылки
from pasing import load_links
for rub in ['2', '3', '4', '40', '41']:
    load_links(rub)

12 2022 ссылки получены
11 2022 ссылки получены
10 2022 ссылки получены
09 2022 ссылки получены
08 2022 ссылки получены
07 2022 ссылки получены
06 2022 ссылки получены
05 2022 ссылки получены
04 2022 ссылки получены
03 2022 ссылки получены
02 2022 ссылки получены
01 2022 ссылки получены
12 2021 ссылки получены
11 2021 ссылки получены
10 2021 ссылки получены
09 2021 ссылки получены
08 2021 ссылки получены
07 2021 ссылки получены
06 2021 ссылки получены
05 2021 ссылки получены
04 2021 ссылки получены
03 2021 ссылки получены
02 2021 ссылки получены
01 2021 ссылки получены
12 2020 ссылки получены
11 2020 ссылки получены
10 2020 ссылки получены
09 2020 ссылки получены
08 2020 ссылки получены
07 2020 ссылки получены
06 2020 ссылки получены
05 2020 ссылки получены
04 2020 ссылки получены
03 2020 ссылки получены
02 2020 ссылки получены
01 2020 ссылки получены
12 2019 ссылки получены
11 2019 ссылки получены
10 2019 ссылки получены
09 2019 ссылки получены
08 2019 ссылки получены
07 2019 ссылки п

In [6]:
# rubrics = {
#     '2': 'politics', '3': 'economics', 
#     '4': 'business', '40':'finance', '41': 'consumer_market'}
# for rub in rubrics.values():
#     urls = []
#     lst = os.listdir(f'komersant_{rub}')
#     lst = [el for el in lst if el[0] != '.']
#     for file in lst:
#         with open(f'komersant_{rub}/{file}') as file:
#             urls_file = file.read()
#             urls_file = urls_file.split('\n')
#             urls.extend(urls_file)

#     urls = np.unique(urls).tolist()

#     with open(f'komersant_{rub}/urls.txt', 'w') as f:
#         for url in urls:
#             f.write(f'{url}\n')

In [20]:
# rubrics = {
#     '2': 'politics', '3': 'economics', 
#     '4': 'business', '40':'finance', '41': 'consumer_market'}
# for rub in rubrics.values():
#     lst = os.listdir(f'komersant_{rub}')
#     lst = [el for el in lst if el[0] != '.']
#     for file in lst:
#         if '20' in file:
#             os.remove(f'komersant_{rub}/{file}')

In [10]:
with open(f'komersant_business/urls.txt') as file:
    urls = file.read()
    urls = urls.split('\n')

In [11]:
data = {'link': [], 'data_or_ex': []}
total_news = len(urls)
current_value = 0

In [13]:
from parsing import parse_from_link_komersant
start_parsing('komersant_business/raw_cites', parse, parse_from_link_komersant, urls, n_threads=2, save_evry=200)

0 saved
